In [19]:
import pandas as pd
import numpy as np

In [20]:

distances = pd.read_csv('dist.csv')
depos = pd.read_csv('depos.csv')
repairs_costs = pd.read_csv('repairs_costs.csv')
stations = pd.read_csv('stations.csv')
vagons_repairs = pd.read_csv('vagons_repairs.csv')
vagons_ways = pd.read_csv('vagons_ways.csv')

all_data = [distances, depos, repairs_costs, stations, vagons_repairs, vagons_ways]
table_names = "distances, depos, repairs_costs, stations, vagons_repairs, vagons_ways".split(", ")

In [21]:
for i in range(len(table_names)):
    print(table_names[i].center(100).upper(), end='\n\n')
    print(all_data[i].head(), end='\n\n\n')

                                             DISTANCES                                              

   station_id      name  depo_id           depo_name  distance_to_depo
0           1  Сосновка        1      Депо-Ленинское             10077
1           1  Сосновка        2  Депо-Комсомольское             10436
2           1  Сосновка        3     Депо-Молодежное             10522
3           2   Липовка        1      Депо-Ленинское              2084
4           2   Липовка        2  Депо-Комсомольское              3789


                                               DEPOS                                                

   id           depo_name
0   1      Депо-Ленинское
1   2  Депо-Комсомольское
2   3     Депо-Молодежное


                                           REPAIRS_COSTS                                            

   id                                 repair_name  repair_cost
0   1                          Капитальный ремонт     94000000
1   2                            Д

In [22]:
closest_depos = distances.loc[distances.groupby('name', as_index=False, sort=False)['distance_to_depo'].idxmin()]

In [23]:
closest_depos.head()

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
3,2,Липовка,1,Депо-Ленинское,2084
7,3,Рожок,2,Депо-Комсомольское,2341
11,4,Гатка,3,Депо-Молодежное,3029
14,5,Георгополь,3,Депо-Молодежное,3662


In [24]:
# стоимость по плану
wagon = 57463085
ways = vagons_ways[vagons_ways['car_num'] == 57463085]
np.array(ways['station_id'])

array([ 2,  6,  9,  5,  8,  2,  6,  9,  3,  6, 10,  3,  7, 10,  4,  8,  1,
        4,  9,  2,  6, 10,  3,  7,  1,  4,  7,  1,  4,  8,  2,  5,  8,  4,
        8,  1,  5,  8,  2,  6,  9,  2,  6, 10,  3,  7, 10,  3,  7,  3,  6,
       10,  3,  7,  1,  4,  7,  1,  5,  8,  2,  5,  9,  2,  8,  1,  5,  9,
        2,  6,  9,  2,  6, 10,  3,  7,  1,  4,  7,  2,  5,  9,  3,  6, 10,
        3,  7, 10,  4,  7,  1,  5,  8,  1,  7,  1,  4,  8,  1,  5,  9,  2,
        5,  9,  2,  6, 10,  3,  6,  2,  6,  9,  3,  6, 10,  4,  7, 10,  4,
        8,  1,  5,  8,  2,  5,  1,  4,  8,  2,  5,  9,  2,  5,  9,  3,  6,
       10,  3,  7, 10,  5,  8,  2,  6,  9,  3,  6, 10,  3,  7, 10,  4,  8,
        1,  4, 10,  3,  7,  1,  4,  8,  2,  5,  8,  2,  5,  9,  3,  6,  9,
        5,  9,  2,  6,  9,  3,  7, 10,  3,  7,  1,  4,  8,  1,  4,  8,  4,
        7,  1,  4,  8,  2,  5,  8,  2,  6,  9,  3,  6, 10,  3,  8,  1,  5,
        9,  2,  6,  9,  2,  6, 10,  3,  7,  1,  4,  7,  3,  6, 10,  4,  7,
        1,  4,  8,  1,  5